In [223]:
from datetime import datetime
import time

import numpy as np
from sklearn.datasets import fetch_california_housing, make_moons
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#make_moons?

In [3]:
x = tf.Variable(3, name='3')
y = tf.Variable(4, name='4')
f = x * x * y + y + 2

In [4]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result= f.eval()
    print(result)

42


In [7]:
init = tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [9]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [10]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    x2.graph is graph
    x2.graph is tf.get_default_graph()

In [12]:
x2.graph is graph

True

In [13]:
x2.graph is tf.get_default_graph()

False

In [14]:
tf.reset_default_graph()

In [15]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(x.eval())
    print(y.eval())

5
10


In [16]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val, z_val)

10 15


In [17]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_eval_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_eval_2 = B.eval(feed_dict={A: [[4, 5, 6], [5, 6, 7]]})
    print(B_eval_1)
    print(B_eval_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [10. 11. 12.]]


In [18]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7465141e+01]
 [ 4.3573415e-01]
 [ 9.3382923e-03]
 [-1.0662201e-01]
 [ 6.4410698e-01]
 [-4.2513184e-06]
 [-3.7732250e-03]
 [-4.2664889e-01]
 [-4.4051403e-01]]


In [19]:
print(m, n)

20640 8


In [20]:
type(housing_data_plus_bias)
housing_data_plus_bias.shape

(20640, 9)

In [21]:
print(m, n)
print(type(m), type(n))

20640 8
<class 'int'> <class 'int'>


In [22]:
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/-run-{}/'.format(root_logdir, now)

In [23]:
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [24]:
target = housing.target.reshape(-1, 1)

In [25]:
def fetch_batch(epoch, batch_index, batch_size):
    X_batch = scaled_housing_data_plus_bias[batch_index * batch_size:(batch_index + 1) * batch_size, :]
    print("fetch_batch, X_batch.shape:", X_batch.shape)
    y_batch = target[batch_index * batch_size:(batch_index + 1) * batch_size]
    return X_batch, y_batch

In [26]:
n_epoch = 1000
learning_rate = 0.01

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [27]:
n_batches

207

In [28]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')

In [ ]:
#X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
#y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

#gradients = 2/m * tf.matmul(tf.transpose(X), error)

gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)
###optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
#training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            saver_path = saver.save(sess, 'saved-models/exp-model.ckpt')
            print("epoch", epoch)
            #print("epoch", epoch, "MSE = ", mse.eval())
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    final_saver_path = saver.save(sess, 'saved-models/exp-model-final.ckpt') 
    print(best_theta)

In [ ]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writter = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [ ]:
#!mkdir saved-models

In [ ]:
!ls saved-models/

In [ ]:
z = tf.constant([1.0, 2.0, 3.0, 4.0], name='z')

In [ ]:
op = tf.math.exp(z)

In [ ]:
op

In [ ]:
with tf.Session() as sess:
    print(op.eval())

In [ ]:
scalar_op = tf.math.exp(2.0)

In [ ]:
with tf.Session() as sess:
    print(scalar_op.eval())

In [96]:
X_moons, y_moons = make_moons(10000)

In [108]:
print(X_moons.shape)
print(y_moons.shape)

(10000, 2)
(10000,)


In [116]:
features_num = X_moons.shape[1]
rows_num = X_moons.shape[0]

In [98]:
#X_moons_plus_bias = np.c_[np.ones((m, 1)), X_moons]

In [230]:
#X_moons_plus_bias.shape
#train_test_split?

In [110]:
#scaler = StandardScaler()
#X_moons_plus_bias_scaled = scaler.fit_transform(X_moons_plus_bias)
X_moons_scaled = scaler.fit_transform(X_moons)

In [231]:
X_moons_scaled_train, X_moons_scaled_test, y_moons_train, y_moons_test= train_test_split(
    X_moons_scaled,
    y_moons,
    test_size=0.2,
    random_state=42    
)

In [232]:
X_moons_scaled_train_tf = tf.convert_to_tensor(X_moons_scaled_train, dtype=tf.float32)
X_moons_scaled_test_tf = tf.convert_to_tensor(X_moons_scaled_test, dtype=tf.float32)
y_moons_train_tf = tf.convert_to_tensor(y_moons_train, dtype=tf.float32)
y_moons_train_tf = tf.reshape(y_moons_train_tf, (y_moons_train_tf.shape[0], 1))
y_moons_test_tf = tf.convert_to_tensor(y_moons_test, dtype=tf.float32)
y_moons_test_tf = tf.reshape(y_moons_test_tf, (y_moons_test_tf.shape[0], 1))

In [114]:
def init_weights(features_num):
    #return np.random.uniform(size=features_num + 1)
    return tf.Variable(tf.random_uniform([features_num + 1, 1], -1, 1), name='weights')

In [121]:
weights = init_weights(features_num)

In [143]:
weights.shape

TensorShape([Dimension(3), Dimension(1)])

In [127]:
print(weights.dtype)
print(type(weights))
print(X_moons_scaled.dtype)
print(type(X_moons))

<dtype: 'float32_ref'>
<class 'tensorflow.python.ops.variables.Variable'>
float64
<class 'numpy.ndarray'>


In [130]:
X_moons_scaled_tf.shape

TensorShape([Dimension(10000), Dimension(2)])

In [131]:
weights.shape

TensorShape([Dimension(3), Dimension(1)])

In [132]:
def sigmoid(x):
    return 1.0 / (1.0 + tf.math.exp(-x))

In [122]:
def linear_calculator(x, weights):
    return tf.tensordot(x, weights[1:], axes=1) + weights[0]

In [138]:
def calculate_errors(y, y_pred):
    return y - y_pred

In [190]:
#linear_calculator(X_moons_scaled_tf, weights)

In [187]:
def calculate_gradient(x, errors):
    #return np.concatenate([np.array([errors.sum()]), np.dot(x.T, errors)])
    #return tf.stack([tf.reshape(tf.reduce_sum(errors), (1, 1)), tf.tensordot(tf.transpose(x), errors, axes=1)], axis=0, name='gradient')
    return tf.concat([tf.reshape(tf.reduce_sum(errors), (1, 1)), tf.tensordot(tf.transpose(x), errors, axes=1)], axis=0, name='gradient')

In [102]:
def gradient(X, y, y_pred):
    return tf.transpose(tf.tensordot(X, y_pred - y, axes=1))

In [228]:
def update_weights(learning_rate, weights, gradient):
    return weights + learning_rate * gradient

In [227]:
def predict(x, weights):
    return tf.where(linear_calculator(x, weights) > 0.0, 1, 0, 'prediction')

In [145]:
y_pred = sigmoid(linear_calculator(X_moons_scaled_tf, weights))
y_pred.shape

TensorShape([Dimension(10000), Dimension(1)])

In [147]:
errors = calculate_errors(y_moons_tf, y_pred)

In [148]:
errors.shape

TensorShape([Dimension(10000), Dimension(1)])

In [191]:
#tf.tensordot(tf.transpose(X_moons_scaled_tf), errors, axes=1)

In [192]:
#tf.tensordot(tf.transpose(X_moons_scaled_tf), errors, axes=0)

In [193]:
#err_sum=tf.reduce_sum(errors)

In [194]:
#tf.reshape(err_sum, (1, 1))

In [183]:
#err_sum_const = tf.constant([err_sum])

In [195]:
#tf.stack?

In [196]:
#gradient = calculate_gradient(X_moons_scaled_tf, errors)

In [95]:
#def gradient(function, x, h):
#    return (function(x + h) - function(x)) / h

In [214]:
#gradient
np.reshape?

In [218]:
def fetch_batch(epoch, batch_index, batch_size):
    X_batch = X_moons_scaled_tf[batch_index * batch_size:(batch_index + 1) * batch_size, :]
    #print("\nfetch_batch, X_batch.shape:", X_batch.shape)
    #print("\nfetch_batch, X_batch.dtype:", X_batch.dtype)
    y_batch = y_moons_tf[batch_index * batch_size:(batch_index + 1) * batch_size]
    return X_batch.eval(), np.reshape(y_batch.eval(), newshape=(batch_size, 1))

In [221]:
n_epoch = 10
learning_rate = 0.01

batch_size = 100
n_batches = int(np.ceil(rows_num / batch_size))

In [198]:
# X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
# ##X = tf.placeholder(tf.float32, shape=(None, n), name='X')
# y = tf.placeholder(tf.float32, shape=(None,), name='y')

# #X = tf.constant([[.2, .7, 0.15], [.4, .4, .5], [.11, .3, .6]], name='X')
# #y = tf.constant([.2, .31, .14], name='y')

# theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name='theta')
# print("\ntheta.get_shape():", theta.get_shape())
# print("tf.shape(theta):", tf.shape(theta))

# y_pred = forward(X, theta)
# print("\nafter call forward")
# print("type(y_pred.get_shape()):", type(y_pred.get_shape()))
# print("y_pred.get_shape():", y_pred.get_shape())
# print("type(tf.shape(y_pred)):", type(tf.shape(y_pred)))
# print("tf.shape(y_pred):", tf.shape(y_pred))
# #error = y_pred - y

# theta_grad = gradient(X, y, y_pred)
# print("\ntheta_grad.get_shape():", theta_grad.get_shape())
# print("tf.shape(theta_grad):", tf.shape(theta_grad))
# training_op = tf.assign(theta, theta - learning_rate * theta_grad)

# init = tf.global_variables_initializer()

# saver = tf.train.Saver()

# with tf.Session() as sess:
#     sess.run(init)
#     for epoch in range(n_epoch):
#         if epoch % 100 == 0:
#             #saver_path = saver.save(sess, 'saved-models/exp-model.ckpt')
#             print("epoch", epoch)
#             #print("epoch", epoch, "MSE = ", mse.eval())
#         for batch_index in range(n_batches):
#             X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
#             #sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#     #best_theta = theta.eval()
#     #final_saver_path = saver.save(sess, 'saved-models/exp-model-final.ckpt') 
#     #print(best_theta)

In [199]:
weights = init_weights(features_num)

In [224]:
X = tf.placeholder(tf.float32, shape=(None, features_num), name='X')

y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

y_pred = sigmoid(linear_calculator(X_moons_scaled_tf, weights))

errors = calculate_errors(y_moons_tf, y_pred)

weights_gradient = calculate_gradient(X_moons_scaled_tf, errors)

training_op = tf.assign(weights, weights - learning_rate * weights_gradient)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

start_time = time.time()

with tf.device('/gpu:0'):
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(n_epoch):
            if epoch % 100 == 0:
                #saver_path = saver.save(sess, 'saved-models/exp-model.ckpt')
                print("epoch", epoch)
                #print("epoch", epoch, "MSE = ", mse.eval())
            for batch_index in range(n_batches):
                X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        best_weights = weights.eval()
        final_saver_path = saver.save(sess, 'saved-models/exp-model-final-{}.ckpt'.format(time.time())) 
        print(best_weights)
print("total time: {}".format((time.time() - start_time) / 60))

epoch 0
[[-5.8779102e+01]
 [-6.5492695e+04]
 [ 7.8963836e+04]]
total time: 88.4780856649081


In [225]:
weights

<tf.Variable 'weights_2:0' shape=(3, 1) dtype=float32_ref>

In [226]:
best_weights

array([[-5.8779102e+01],
       [-6.5492695e+04],
       [ 7.8963836e+04]], dtype=float32)

In [ ]:
##gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)
###optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
#training_op = optimizer.minimize(mse)

##X = tf.placeholder(tf.float32, shape=(None, n), name='X')
##y = tf.placeholder(tf.float32, shape=(None,), name='y')

#X = tf.constant([[.2, .7, 0.15], [.4, .4, .5], [.11, .3, .6]], name='X')
#y = tf.constant([.2, .31, .14], name='y')

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name='theta')
print("\ntheta.get_shape():", theta.get_shape())
print("tf.shape(theta):", tf.shape(theta))

y_pred = forward(X, theta)
print("\nafter call forward")
print("type(y_pred.get_shape()):", type(y_pred.get_shape()))
print("y_pred.get_shape():", y_pred.get_shape())
print("type(tf.shape(y_pred)):", type(tf.shape(y_pred)))
print("tf.shape(y_pred):", tf.shape(y_pred))
#error = y_pred - y

theta_grad = gradient(X, y, y_pred)
print("\ntheta_grad.get_shape():", theta_grad.get_shape())
print("tf.shape(theta_grad):", tf.shape(theta_grad))
#training_op = tf.assign(theta, theta - learning_rate * theta_grad)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            #saver_path = saver.save(sess, 'saved-models/exp-model.ckpt')
            print("epoch", epoch)
            #print("epoch", epoch, "MSE = ", mse.eval())
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            #sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    #best_theta = theta.eval()
    #final_saver_path = saver.save(sess, 'saved-models/exp-model-final.ckpt') 
    #print(best_theta)

In [ ]:
!ls saved-models/

In [ ]:
saver = tf.train.import_meta_graph('saved-models/exp-model-final.ckpt.meta')

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, 'saved-models/exp-model-final.ckpt.meta')

In [31]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_uniform(w_shape), name='weigths')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0., name='relu')

In [32]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for _ in range(5)]
output = tf.add_n(relus, name='output')

In [33]:
output

<tf.Tensor 'output:0' shape=(?, 1) dtype=float32>

In [37]:
# from sklearn.datasets import fetch_mldata
# mnist = fetch_mldata('MNIST original')
# mnist
from sklearn.datasets import load_digits

In [38]:
digits = load_digits()

In [39]:
type(digits)

sklearn.utils.Bunch

In [40]:
#images_and_labels = list(zip(digits.images, digits.target))

In [44]:
images = digits.images
targets = digits.target

In [57]:
reshaped_images = np.reshape(images, newshape=(images.shape[0], images.shape[1] * images.shape[2]))

In [58]:
reshaped_images.shape

(1797, 64)

In [59]:
scaled_reshaped_images = scaler.fit_transform(reshaped_images)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(
    scaled_reshaped_images,
    targets,
    test_size=0.25,
    random_state=42
)

In [62]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(
    hidden_units=(300, 100),
    n_classes=10,
    feature_columns=feature_columns
)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)
dnn_clf.fit(X_train, y_train, batch_size=50, steps=40000)

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc8640b8470>, '_master': '',

INFO:tensorflow:loss = 0.0022905567, step = 1900 (0.171 sec)
INFO:tensorflow:global_step/sec: 575.746
INFO:tensorflow:loss = 0.0015257047, step = 2000 (0.174 sec)
INFO:tensorflow:global_step/sec: 548.241
INFO:tensorflow:loss = 0.00091822003, step = 2100 (0.183 sec)
INFO:tensorflow:global_step/sec: 572.653
INFO:tensorflow:loss = 0.0007168489, step = 2200 (0.175 sec)
INFO:tensorflow:global_step/sec: 525.415
INFO:tensorflow:loss = 0.0013131395, step = 2300 (0.190 sec)
INFO:tensorflow:global_step/sec: 569.324
INFO:tensorflow:loss = 0.00046992346, step = 2400 (0.175 sec)
INFO:tensorflow:global_step/sec: 569.188
INFO:tensorflow:loss = 0.0009340857, step = 2500 (0.176 sec)
INFO:tensorflow:global_step/sec: 553.546
INFO:tensorflow:loss = 0.0011635409, step = 2600 (0.181 sec)
INFO:tensorflow:global_step/sec: 541.979
INFO:tensorflow:loss = 0.0007821798, step = 2700 (0.185 sec)
INFO:tensorflow:global_step/sec: 533.426
INFO:tensorflow:loss = 0.000736103, step = 2800 (0.188 sec)
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 531.669
INFO:tensorflow:loss = 0.0001643242, step = 10000 (0.188 sec)
INFO:tensorflow:global_step/sec: 557.566
INFO:tensorflow:loss = 7.294329e-05, step = 10100 (0.179 sec)
INFO:tensorflow:global_step/sec: 514.033
INFO:tensorflow:loss = 0.0002483264, step = 10200 (0.195 sec)
INFO:tensorflow:global_step/sec: 538.47
INFO:tensorflow:loss = 0.0002468462, step = 10300 (0.185 sec)
INFO:tensorflow:global_step/sec: 550.091
INFO:tensorflow:loss = 0.00032476, step = 10400 (0.182 sec)
INFO:tensorflow:global_step/sec: 565.119
INFO:tensorflow:loss = 0.00015859598, step = 10500 (0.177 sec)
INFO:tensorflow:global_step/sec: 481.374
INFO:tensorflow:loss = 4.620325e-05, step = 10600 (0.208 sec)
INFO:tensorflow:global_step/sec: 548.403
INFO:tensorflow:loss = 0.00018691093, step = 10700 (0.182 sec)
INFO:tensorflow:global_step/sec: 568.134
INFO:tensorflow:loss = 8.43733e-05, step = 10800 (0.176 sec)
INFO:tensorflow:global_step/sec: 533.819
INFO:tensorflow:loss = 8.9134875e-

INFO:tensorflow:loss = 7.911937e-05, step = 17900 (0.194 sec)
INFO:tensorflow:global_step/sec: 553.723
INFO:tensorflow:loss = 0.00010502185, step = 18000 (0.180 sec)
INFO:tensorflow:global_step/sec: 538.935
INFO:tensorflow:loss = 4.2893575e-05, step = 18100 (0.186 sec)
INFO:tensorflow:global_step/sec: 583.34
INFO:tensorflow:loss = 6.436221e-05, step = 18200 (0.173 sec)
INFO:tensorflow:global_step/sec: 575.353
INFO:tensorflow:loss = 7.527436e-05, step = 18300 (0.172 sec)
INFO:tensorflow:global_step/sec: 593.155
INFO:tensorflow:loss = 4.636864e-05, step = 18400 (0.168 sec)
INFO:tensorflow:global_step/sec: 523.771
INFO:tensorflow:loss = 8.920743e-05, step = 18500 (0.191 sec)
INFO:tensorflow:global_step/sec: 555.823
INFO:tensorflow:loss = 9.369578e-05, step = 18600 (0.180 sec)
INFO:tensorflow:global_step/sec: 554.007
INFO:tensorflow:loss = 5.0524188e-05, step = 18700 (0.181 sec)
INFO:tensorflow:global_step/sec: 585.552
INFO:tensorflow:loss = 3.8318638e-05, step = 18800 (0.171 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 564.509
INFO:tensorflow:loss = 6.3590865e-05, step = 25900 (0.178 sec)
INFO:tensorflow:global_step/sec: 585.782
INFO:tensorflow:loss = 4.948205e-05, step = 26000 (0.170 sec)
INFO:tensorflow:global_step/sec: 575.107
INFO:tensorflow:loss = 4.897676e-05, step = 26100 (0.174 sec)
INFO:tensorflow:global_step/sec: 535.837
INFO:tensorflow:loss = 4.2353993e-05, step = 26200 (0.187 sec)
INFO:tensorflow:global_step/sec: 577.611
INFO:tensorflow:loss = 4.1285602e-05, step = 26300 (0.173 sec)
INFO:tensorflow:global_step/sec: 546.647
INFO:tensorflow:loss = 5.422444e-05, step = 26400 (0.183 sec)
INFO:tensorflow:global_step/sec: 578.025
INFO:tensorflow:loss = 3.0054436e-05, step = 26500 (0.173 sec)
INFO:tensorflow:global_step/sec: 572.056
INFO:tensorflow:loss = 3.576442e-05, step = 26600 (0.175 sec)
INFO:tensorflow:global_step/sec: 485.116
INFO:tensorflow:loss = 7.0971255e-05, step = 26700 (0.206 sec)
INFO:tensorflow:global_step/sec: 565.74
INFO:tensorflow:loss = 1.254

INFO:tensorflow:loss = 2.6027996e-05, step = 33800 (0.188 sec)
INFO:tensorflow:global_step/sec: 551.909
INFO:tensorflow:loss = 3.4155557e-05, step = 33900 (0.181 sec)
INFO:tensorflow:global_step/sec: 536.924
INFO:tensorflow:loss = 4.0612464e-05, step = 34000 (0.186 sec)
INFO:tensorflow:global_step/sec: 551.12
INFO:tensorflow:loss = 1.8749877e-05, step = 34100 (0.182 sec)
INFO:tensorflow:global_step/sec: 518.875
INFO:tensorflow:loss = 5.2686013e-05, step = 34200 (0.193 sec)
INFO:tensorflow:global_step/sec: 176.489
INFO:tensorflow:loss = 2.9732875e-05, step = 34300 (0.567 sec)
INFO:tensorflow:global_step/sec: 568.543
INFO:tensorflow:loss = 4.7785517e-05, step = 34400 (0.176 sec)
INFO:tensorflow:global_step/sec: 389
INFO:tensorflow:loss = 3.1628224e-05, step = 34500 (0.258 sec)
INFO:tensorflow:global_step/sec: 314.889
INFO:tensorflow:loss = 2.2802442e-05, step = 34600 (0.317 sec)
INFO:tensorflow:global_step/sec: 474.765
INFO:tensorflow:loss = 4.0271967e-05, step = 34700 (0.210 sec)
INFO:t

SKCompat()

In [83]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
    return X_batch, y_batch

In [63]:
from sklearn.metrics import accuracy_score

In [64]:
y_pred = dnn_clf.predict(X_test)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpiwludibt/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [65]:
accuracy_score(y_test, y_pred['classes'])

0.9844444444444445

In [67]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [68]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None, ), name='y')

In [69]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name='kernel')
        b = tf.Variable(tf.zeros([n_neurons]), name='bias')
        Z = tf.matmul(X, W) + b
        if activation:
            return activation(Z)
        else:
            return Z

In [76]:
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name='outputs')

In [79]:
# with tf.name_scope('dnn1'):
#     hidden11 = tf.layers.dense(X, n_hidden1, name='hidden11', activation=tf.nn.relu)
#     hidden12 = tf.layers.dense(hidden1, n_hidden2, name='hidden12', activation=tf.nn.relu)
#     logits1 = tf.layers.dense(hidden2, n_outputs, name='outputs1')

In [80]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='logits')

In [81]:
learning_rate = 0.01

In [85]:
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [86]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [87]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [88]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [90]:
!pwd

/home/uldo/work/kaggle/competitions/APTOS-2019-Blindness-Detection/code/tf-py-torch-tutorials


In [89]:
n_epochs = 40
batch_size = 50

In [93]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, " Train accuracy: ", acc_train, " Test accuracy: ", acc_test)
    save_path = saver.save(sess, './my_model_final.cpkt')

0  Train accuracy:  0.94  Test accuracy:  0.9107
1  Train accuracy:  0.92  Test accuracy:  0.9296
2  Train accuracy:  0.94  Test accuracy:  0.9395
3  Train accuracy:  0.88  Test accuracy:  0.9448
4  Train accuracy:  0.94  Test accuracy:  0.949
5  Train accuracy:  0.98  Test accuracy:  0.9506
6  Train accuracy:  0.98  Test accuracy:  0.9558
7  Train accuracy:  0.98  Test accuracy:  0.9594
8  Train accuracy:  0.98  Test accuracy:  0.9618
9  Train accuracy:  0.92  Test accuracy:  0.9616
10  Train accuracy:  0.98  Test accuracy:  0.9645
11  Train accuracy:  0.96  Test accuracy:  0.9664
12  Train accuracy:  0.98  Test accuracy:  0.9677
13  Train accuracy:  1.0  Test accuracy:  0.9684
14  Train accuracy:  1.0  Test accuracy:  0.9688
15  Train accuracy:  1.0  Test accuracy:  0.9699
16  Train accuracy:  0.98  Test accuracy:  0.9704
17  Train accuracy:  1.0  Test accuracy:  0.9695
18  Train accuracy:  0.96  Test accuracy:  0.9711
19  Train accuracy:  1.0  Test accuracy:  0.9724
20  Train accura

In [94]:
np.random.uniform?